# Next.js Middleware

## Introduction to Middleware
Middleware in Next.js intercepts incoming requests before they reach their destination, allowing modification of requests or responses. It runs before cached content and route matching, providing access to the **Request object**, **Response object**, and a **next() function** during the request-response cycle. This concept builds on Express.js middleware but is tailored for Next.js App Router applications.  

Key benefits include adding custom headers, authentication checks, redirects, or early error responses (e.g., "Database down, try later"). Create a `middleware.js` (or `.ts`) file at the project root for App Router setups. 

## Creating a Middleware File
Generate a Next.js project with `npx create-next-app@latest middleware --no-typescript`, then add `middleware.js` at the root level. Import `NextResponse` from `'next/server'` for response manipulation.  

**Basic Template:**
```javascript
import { NextResponse } from 'next/server';

export function middleware(request) {
  // Logic here
  return NextResponse.next(); // Proceed normally
}
```
The file must export a `middleware` function. Documentation provides starter templates; copy for efficiency.  

## Path Matching with Config
By default, middleware runs on all routes, which can cause infinite loops (e.g., redirecting `/` to itself). Use `export const config = { matcher: '/path' }` to target specific paths. 

**Matcher Syntax:**
- **Single path:** `matcher: '/about/:path*'` matches `/about` and subpaths like `/about/team`.
- **Multiple paths:** `matcher: ['/about/:path*', '/dashboard/:path*']`.
- **Regex:** `'/((?!api|_next/static|_next/image|favicon.ico).*)'` excludes static/API files  .
- **Modifiers:** `:path*` (zero or more), `:path?` (zero or one), `:path+` (one or more).

Matchers must be static constants starting with `/` and support named parameters.  

**Alternative:** Use conditional checks like `if (request.nextUrl.pathname.startsWith('/about'))` inside middleware—no config needed. 

## NextResponse Methods
**NextResponse** enables key operations on requests/responses. 

| Method | Purpose | Example |
|--------|---------|---------|
| `NextResponse.redirect(url)` | Changes URL and navigates browser (visible redirect). | `NextResponse.redirect(new URL('/', request.url))` sends `/about` to `/`  . |
| `NextResponse.rewrite(url)` | Serves content from another path without URL change (invisible proxy). | Rewrite `/about` to show `/about-2` content but keep `/about` URL   . |
| `NextResponse.json(data)` | Returns custom JSON, halting further processing. | `NextResponse.json({ message: 'Hello from middleware' })` overrides page content  . |
| `NextResponse.next()` | Proceeds to destination unchanged. | Default for non-matching paths  . |

**Redirect vs Rewrite:** Redirect alters the URL bar; rewrite masks it (e.g., `/dashboard` shows `/` content without changing address).  

## Practical Examples
- **Redirect `/about` to `/home`:** 
  ```javascript
  export const config = { matcher: '/about/:path*' };
  export function middleware(request) {
    return NextResponse.redirect(new URL('/home', request.url));
  }
  ```
  Any `/about/...` redirects visibly. 

- **Rewrite `/dashboard` to `/`:** 
  ```javascript
  if (request.nextUrl.pathname.startsWith('/dashboard')) {
    return NextResponse.rewrite(new URL('/', request.url));
  }
  ```
  `/dashboard` displays home content invisibly. 

- **Custom Headers:** `response.headers.set('x-custom-header', 'value')` adds headers before proceeding.  

- **Early Response:** Return JSON for errors, bypassing pages. 

Avoid loops by excluding paths in matcher; test with `npm run dev`. 

## Additional Features
- **Cookies:** Access via `request.cookies.get('name')`; set with `response.cookies.set('name', 'value')`. 
- **Headers:** Clone `new Headers(request.headers)` and modify. 
- **Runtime:** Edge runtime by default for speed. 

Refer to official docs for cookies/headers details. 

## Key Takeaways
Next.js middleware enables request interception for redirects, rewrites, headers, and custom responses via `middleware.js` and `NextResponse`. Use **matchers** for path targeting and distinguish **redirect** (URL changes) from **rewrite** (content proxy). Ideal for auth, logging, or optimizations; always exclude static/API paths to prevent issues.